In [ ]:
#import the modules
import pandas as pd
import requests
import json
import csv
import time
import datetime
from pandas.io.json import json_normalize

In [ ]:
#Get an idea of the volume of the posts data from 03/01/2020 @ 12:00am (UTC) to 06/01/2020 @ 12:00am (UTC)

url = "https://api.pushshift.io/reddit/search/submission"

requests.get(url, params = {"subreddit": "teenagers", "size": 0, "before": 1590969600, "after":1583020800, "aggs" : "subreddit"}).json()["aggs"]


{'subreddit': [{'doc_count': 525066, 'key': 'teenagers'}]}

In [ ]:
#Get an idea of the volume of the comments data from 03/01/2020 @ 12:00am (UTC) to 06/01/2020 @ 12:00am (UTC)

url = "https://api.pushshift.io/reddit/search/comment"

requests.get(url, params = {"subreddit": "teenagers", "size": 0, "before": 1590969600, "after":1583020800, "aggs" : "subreddit"}).json()["aggs"]


{'subreddit': [{'doc_count': 4589594, 'key': 'teenagers'}]}

In [ ]:
import random
PUSHSHIFT_REDDIT_URL = "http://api.pushshift.io/reddit/"

def fetchObjects(**kwargs):
    # Default paramaters for API query
    params = {
        "sort_type":"created_utc",
        "sort":"asc",
        "size":1000,
        "before": 1590969600 
        }

    # Add additional paramters based on function arguments
    for key,value in kwargs.items():
        params[key] = value

    # Print API query paramaters
    print(params)

    # Set the type variable based on function input
    # The type can be "comment" or "submission", default is "comment"
    type = "comment"
    if 'type' in kwargs and kwargs['type'].lower() == "submission":
        type = "submission"
    
    # Perform an API request
    r = requests.get(PUSHSHIFT_REDDIT_URL + "search/" + type + "/", params=params, timeout=30)

    # Check the status code, if successful, process the data
    if r.status_code == 200:
        response = json.loads(r.text)
        data = response['data']
        sorted_data_by_id = sorted(data, key=lambda x: int(x['id'],36))
        return sorted_data_by_id

def extract_reddit_data(**kwargs):
    # Speficify the start timestamp
    max_created_utc = 1583020800
    max_created_utc = 1590954785
    max_id = 0

    # Open a file for JSON output
    file = open("submissions.json","a")

    # While loop for recursive function
    while 1:
        nothing_processed = True
        # Call the recursive function
        objects = fetchObjects(**kwargs,after=max_created_utc)
        
        # Loop the returned data, ordered by date
        for object in objects:
            id = int(object['id'],36)
            if id > max_id:
                nothing_processed = False
                created_utc = object['created_utc']
                max_id = id
                if created_utc > max_created_utc: max_created_utc = created_utc
                # Output JSON data to the opened file
                print(json.dumps(object,sort_keys=True,ensure_ascii=True),file=file)
        
        # Exit if nothing happened
        if nothing_processed: return

        # Sleep a little before the next recursive function call
        time.sleep(random.uniform(3,20))



In [ ]:
# Start program by calling function with:
# 1) Subreddit specified
# 2) The type of data required (comment or submission)
extract_reddit_data(subreddit="teenagers",type="submission") #query API for posts

{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'before': 1590969600, 'subreddit': 'teenagers', 'type': 'submission', 'after': 1590954785}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'before': 1590969600, 'subreddit': 'teenagers', 'type': 'submission', 'after': 1590968935}
{'sort_type': 'created_utc', 'sort': 'asc', 'size': 1000, 'before': 1590969600, 'subreddit': 'teenagers', 'type': 'submission', 'after': 1590969597}


In [ ]:
data = []
keys = ["created_utc","title","selftext","subreddit"]
count_json_err = 0
count_deleted_author = 0

with open('./submissions.json') as f:
    line = f.readline()
    while line:
      has_selftext = True
      try: 
        json_data = json.loads(line)
      except:
        count_json_err += 1
        print(line)
        line = f.readline()
        continue
      d = {"created_utc": None,"title": None,"selftext": None,"subreddit": None}
      for key in d.keys():
        if not key in json_data:
          has_selftext = False
          count_deleted_author += 1
          break
        try:
          d[key] = json_data[key].replace("\n"," ").replace("\t"," ")
        except:
          d[key] = json_data[key]
      if has_selftext: data.append(d)
      line = f.readline()

with open('submissions_teenagers.txt', 'w') as f:
    f.write("%s\t%s\t%s\t%s\n"%(keys[0], keys[1], keys[2], keys[3]))
    for d in data:
        f.write("%s\t%s\t%s\t%s\n"%(d[keys[0]],d[keys[1]],d[keys[2]],d[keys[3]]))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{"all_awardings": [], "allow_live_comments": false, "author": "Memeisseur_", "author_flair_css_class": null, "author_flair_richtext": [], "author_flair_text": null, "author_flair_type": "text", "author_fullname": "t2_4eu6eugi", "author_patreon_flair": false, "author_premium": false, "awarders": [], "can_mod_post": false, "contest_mode": false, "created_utc": 1584379131, "domain": "i.redd.it", "full_link": "https://www.reddit.com/r/teenagers/comments/fjojrl/crus_cant_say_that_is_now_in_my_province_just/", "gildings": {}, "id": "fjojrl", "is_crosspostable": true, "is_meta": false, "is_original_content": false, "is_reddit_media_domain": true, "is_robot_indexable": true, "is_self": false, "is_video": false, "link_flair_background_color": "", "link_flair_richtext": [], "link_flair_text_color": "dark", "link_flair_type": "text", "locked": false, "media_only": false, "no_follow": true, "num_comments": 0, "num_crossposts": 0, "over_18": false, "parent_whitelist_status": "all_ads", "permalink":

In [ ]:
print(len(data))
print(count_json_err)
print(count_deleted_author)

521204
49
3800


Not sure why some data disappeared. But it was a very small portion of the total subreddit posts (525066).